# <center style="font-family: consolas; font-size: 32px; font-weight: bold;">  Prompt Engineering for Instruction-Tuned LLM: Text Summarization & Information Retrieval </center>
***


# 


LLMs offer a revolutionary approach by enabling the execution of various tasks with a single prompt, streamlining the traditional workflow that involves developing and deploying separate models for distinct objectives.

Through practical examples, the article illustrates the efficiency of LLMs in tasks such as sentiment analysis of product reviews, identification of emotions, and extraction of valuable information like product and company names from customer reviews. 

The versatility of LLMs is further demonstrated as they seamlessly perform multiple tasks concurrently through unified prompts. The article also delves into more complex natural language processing tasks, including topic inference and indexing topics for news articles. 

Overall, the transformative impact of prompt engineering with LLMs is highlighted, presenting an efficient and powerful tool for both seasoned machine learning developers and those entering the field.


#### <a id="top"></a>


# <div style="box-shadow: rgb(60, 121, 245) 0px 0px 0px 3px inset, rgb(255, 255, 255) 10px -10px 0px -3px, rgb(31, 193, 27) 10px -10px, rgb(255, 255, 255) 20px -20px 0px -3px, rgb(255, 217, 19) 20px -20px, rgb(255, 255, 255) 30px -30px 0px -3px, rgb(255, 156, 85) 30px -30px, rgb(255, 255, 255) 40px -40px 0px -3px, rgb(255, 85, 85) 40px -40px; padding:20px; margin-right: 40px; font-size:30px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(60, 121, 245);"><b>Table of contents</b></div>

<div style="background-color: rgba(60, 121, 245, 0.03); padding:30px; font-size:15px; font-family: consolas;">
<ul>
    <li><a href="#1" target="_self" rel=" noreferrer nofollow">1. Getting Started & Setting Working Environment </a> </li>
    <li><a href="#2" target="_self" rel=" noreferrer nofollow">2. Sentiment Analysis of Product Review </a></li>
    <li><a href="#3" target="_self" rel=" noreferrer nofollow">3.  Identify Emotions </a></li> 
    <li><a href="#4" target="_self" rel=" noreferrer nofollow">4. Doing Multiple Tasks at Once </a></li>
        <li><a href="#5" target="_self" rel=" noreferrer nofollow">5. Extract Product & Company Names From Customer Reviews </a></li>
            <li><a href="#6" target="_self" rel=" noreferrer nofollow">6. Topics Inferring </a></li>
            <li><a href="#7" target="_self" rel=" noreferrer nofollow">7. Make a News Alert for Certain Topics </a></li>
            <li><a href="#8" target="_self" rel=" noreferrer nofollow">8. Conclusion </a></li>
                <li><a href="#9" target="_self" rel=" noreferrer nofollow">9. References </a></li>



    
    
</ul>
</div>

***



<a id="1"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 1. Getting Started & Setting Working Environment </b></div>


In the traditional machine learning workflow, you’d have to collect the label data set, train a model, figure out how to deploy the model somewhere in the cloud, and make inferences. This could work pretty well, but it was a lot of work to go through that process. Also for every task, such as sentiment, extracting names, and classifying emotions, you will have to train and deploy a separate model. 

One of the nice things about the large language models is that you can write a prompt for many tasks like these and have it start generating results immediately. This gives tremendous speed in terms of application development. And you can also just use one model, one API to do many different tasks rather than needing to figure out how to train and deploy a lot of different models.

Let's initialize our working environment as usual by importing the OpenAI package and then define the API key from the local .env file:

In [1]:
!pip install openai
from openai import OpenAI
import openai
import os
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
openai.api_key = user_secrets.get_secret("openai_api")
client = OpenAI(
    # This is the default and can be omitted
    api_key=openai.api_key,
)

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/5d/12/d445f6b7309600bcd7f895495603d898a31ee35345094f8e16fd9b5e8718/openai-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/39/9b/4937d841aee9c2c8102d9a4eeb800c7dad25386caabb4a1bf5010df81a57/httpx-0.26.0-py3-none-any.whl.metadata
  Obtaining dependency information for typing-extensions<5,>=4.7 from https://files.pythonhosted.org/packages/b7/f4/6a90020cd2d93349b442bfcb657d0dc91eee65491600b2cb1d388bc98e6b/typing_extensions-4.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpcore==1.* from https://files.pythonhosted.org/packages/56/ba/78b0a99c4da0ff8b0f59defa2f13ca4668189b134bd9840b6202a93d9a0f/httpcore-1.0.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.0 MB/s eta 0:00:00
   ━━

Now let's define a function that will take the prompt and return the response from the LLM. 

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content


<a id="2"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 2. Sentiment Analysis of Product Review </b></div>


Sentiment analysis is a natural language processing (NLP) technique used to determine the sentiment expressed in a piece of text. When it comes to product reviews, sentiment analysis can be valuable for businesses to understand customer opinions and feedback. In this article, we will use a review about a product which will be a lamp. Here is the review.

In [3]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

Let's write a prompt to classify the sentiment of this review. I want the system to tell me what is the sentiment. We will ask it “What is the sentiment of the following product review” with the usual delimiter and we will then run that. 

In [4]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the product review is positive.


It says, **The sentiment of the product review is positive.**, which actually, seems pretty right.

If you want to give a more concise response to make it easier for post-processing, I can take this prompt and add another instruction to give you answers to a single word, either positive or negative. So it just prints out positively like this, which makes it easier for a piece of text to take this output process it, and do something with it.



In [5]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)


positive


<a id="3"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 3. Identify Emotions </b></div>

Let’s take another use case where we can use prompt engineering to do a complex sentiment task which is to identify emotions. So, large language models are pretty good at extracting specific things out of a piece of text. In this case, we’re expressing the emotions and this could be useful for understanding how your customers think about a particular product. For a lot of customer support organizations, it’s important to understand if a particular user is extremely upset. Here is the prompt to do this:

In [6]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)


satisfied, pleased, grateful, impressed, happy


You might have a different classification problem such as identifying a c certain emotion. So I would like to know whether the customer is angry or not. If someone is really angry, it might merit paying extra attention to have a customer review, to have customer support or customer success, to reach out to figure out what’s going on and make things right for the customer.

In [7]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)


No


In this case, the customer is not angry. Notice that with supervised learning, if I had wanted to build all of these classifiers, there’s no way I would have been able to do this with supervised learning in just a few minutes.

<a id="4"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 4. Extract Product & Company Names From Customer Reviews </b></div>


Information extraction is a part of NLP that relates to taking a piece of text and extracting certain things that you want to know from the text. So in this prompt, I will ask the LLM to identify the item purchased and the name of the company that made the item. 

If you are trying to summarize many reviews from an online shopping e-commerce website, it might be useful for your large collection of reviews to figure out what were the items, who made the item, and the positive and negative sentiments, to track trends about positive or negative sentiment for specific items or for specific manufacturers. In this example, I’m going to ask it to format your response as a JSON object with “**Item**” and “**Brand**” as the keys.



In [8]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Item": "lamp",
  "Brand": "Lumina"
}


<a id="5"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 5. Doing Multiple Tasks at Once </b></div>



In the examples we’ve gone through, we have written prompts to recognize the sentiment, figure out if someone is angry, and then also extract the item and the brand. 

One way to extract all of this information would be to use three or four prompts and call the “get_completion” function, three times or four times to extract these different views one at a time. But it turns out you can write a single prompt to extract all of this information at the same time.

So let’s say “identify the following items, extract sentiment, is the reviewer expressing anger, item purchased, company that made it”. Then I’m going to tell it to format the anger value as a boolean value. 

In [9]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)


{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp",
  "Brand": "Lumina"
}


<a id="6"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 6. Topics Inferring </b></div>

Another complex NLP task that you can do using is topic inferring. Given a long piece of text, you know, what is this piece of text about? What are the topics? 

Here’s a fictitious newspaper article about how government workers feel about the agency they work for. So, the recent survey conducted by the government.



In [10]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""


So, given an article like this, we can ask it, with this prompt, to determine five topics that are being discussed in the following text. Let’s make each item one or two words long, for my response, in a comma-separated list.

In [11]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

1. Government survey
2. Department satisfaction rating
3. NASA
4. Social Security Administration
5. Job satisfaction improvement


It’s about a government survey, it’s about job satisfaction, it’s about NASA, and so on. So, overall, I think, a pretty nice extraction of a list of topics. 

<a id="7"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 7. Make Alert for Certain Topics
 </b></div>

 
 
 If you have a collection of articles and extract topics, you can then also use a large language model to help you index into different topics. So, let me use a slightly different topic list.



In [12]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

In [13]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

[1, 0, 0, 1, 1]


In machine learning, this is sometimes called a “Zero-Shot Learning Algorithm”, because we didn’t give it any training data that was labeled, so that’s Zero-Shot. With just a prompt, it was able to determine which of these topics were covered in that news article. So, if you want to generate a news alert, say, so that process news, and I like a lot of work that NASA does. So, if you want to build a system that can take this, put this information into a dictionary, and whenever NASA news pops up, print “ALERT: New NASA story!”, they can use this to very quickly take any article, figure out what topics it is about, and if the topic includes NASA, have it print out “ALERT: New NASA story!”. 

This prompt that I use up here isn’t very robust. If I wanted a production system, I would probably have it output the answer in JSON format, rather than as a list, because the output of the large language model can be a little bit inconsistent. So, this is a pretty brittle piece of code.

<a id="8"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 8. Conclusion </b></div>

In conclusion in just a few minutes, you can build multiple systems for making inferences about text that previously would have taken days or even weeks for a skilled machine-learning developer. 

This is very exciting that both for skilled machine learning developers, as well as for people who are newer to machine learning, you can now use prompting to very quickly build and start making inferences on pretty complicated natural language processing tasks like these.

<a id="9"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 9. References  </b></div>
